# imports

In [ ]:
import networkx as nx
import sqlalchemy as sa
from tqdm.notebook import tqdm

import src
from src.bokeh_graph import BokehGraph
from src.connect import create_sqlite_session
from src.models import Author

# setup

In [ ]:
engine, s = create_sqlite_session(src.PATH / "data/example.db")

# Erstelle Netzwerk

In [ ]:
# Query alle Autoren
authors = s.query(Author).options(sa.orm.subqueryload(Author.items))

In [ ]:
def add_node(author, net):
    node = author.pk_authors

    if not graph.has_node(node):
        graph.add_node(
            node,
            firstname=author.firstname,
            lastname=author.lastname,
            n_items=len(author.items),
        )

    return node

In [ ]:
graph = nx.Graph()

for author in tqdm(authors, total=authors.count()):
    # add node if not exists
    node = add_node(author, graph)

    # add coauthors and edges that do not exist yet
    for coauthor in author.coauthors:
        co_node = add_node(coauthor, graph)

        if not graph.has_edge(node, co_node):
            graph.add_edge(node, co_node)

  0%|          | 0/16546 [00:00<?, ?it/s]

## Bereinige Netzwerk, wähle größte Komponente, berechne Netzwerkstatistiken

In [ ]:
isolates = nx.isolates(graph)

nodes = list(graph.nodes)
for node in nodes:
    if node in isolates:
        graph.remove_node(node)

In [ ]:
len(graph.nodes)

16546

In [ ]:
largest_components = sorted(nx.connected_components(graph), key=len, reverse=True)
subgraph = graph.subgraph(largest_components[1])

In [ ]:
degree = nx.degree(subgraph)
degree = dict(degree)
nx.set_node_attributes(subgraph, degree, "degree")

In [ ]:
bet = nx.betweenness_centrality(subgraph)
bet = dict(bet)
nx.set_node_attributes(subgraph, bet, "betweeness_centrality")

# Plotte Netzwerk

In [ ]:
# Falls der Plot in eine extra Datei geschrieben werden soll:
from bokeh.io import output_file

(src.PATH / "tmp").mkdir(exist_ok=True)
output_file(src.PATH / "tmp/myoutput.html")

In [ ]:
# inline sagt, ob Plot im Notebook angezeigt werden soll (Daten werden dann auch im NB gespeichert, wie bei altair)
plot = BokehGraph(subgraph, width=1024, height=768, inline=False)

In [ ]:
plot.layout()

In [ ]:
plot.draw(
    color_by="degree", palette="viridis", edge_alpha=0.01, max_colors=5, node_size=5
)